#1. Install Necessary Packages

In [ ]:
# Install necessary packages
!pip install ultralytics
!pip install opencv-python
!pip install ffmpeg-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 898.8/898.8 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.3/491.3 kB 9.1 MB/s eta 0:00:00


#2. Importing Libraries and Initializing the Model

Note:

the below cell is to confirm that which Class ID corresponds to car to truck

In [ ]:
from ultralytics import YOLO
import cv2
from google.colab import files
import os
from collections import deque

# Load the YOLO model (ensure the model file is correctly named and accessible)
model = YOLO("yolo11x.pt")  # Replace with your model path if different

# Verify class names and IDs
print("Model Classes and IDs:")
for class_id, class_name in model.names.items():
    print(f"ID {class_id}: {class_name}")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 109M/109M [00:00<00:00, 282MB/s]


Model Classes and IDs:
ID 0: person
ID 1: bicycle
ID 2: car
ID 3: motorcycle
ID 4: airplane
ID 5: bus
ID 6: train
ID 7: truck
ID 8: boat
ID 9: traffic light
ID 10: fire hydrant
ID 11: stop sign
ID 12: parking meter
ID 13: bench
ID 14: bird
ID 15: cat
ID 16: dog
ID 17: horse
ID 18: sheep
ID 19: cow
ID 20: elephant
ID 21: bear
ID 22: zebra
ID 23: giraffe
ID 24: backpack
ID 25: umbrella
ID 26: handbag
ID 27: tie
ID 28: suitcase
ID 29: frisbee
ID 30: skis
ID 31: snowboard
ID 32: sports ball
ID 33: kite
ID 34: baseball bat
ID 35: baseball glove
ID 36: skateboard
ID 37: surfboard
ID 38: tennis racket
ID 39: bottle
ID 40: wine glass
ID 41: cup
ID 42: fork
ID 43: knife
ID 44: spoon
ID 45: bowl
ID 46: banana
ID 47: apple
ID 48: sandwich
ID 49: orange
ID 50: broccoli
ID 51: carrot
ID 52: hot dog
ID 53: pizza
ID 54: donut
ID 55: cake
ID 56: chair
ID 57: couch
ID 58: potted plant
ID 59: bed
ID 60: dining table
ID 61: toilet
ID 62: tv
ID 63: laptop
ID 64: mouse
ID 65: remote
ID 66: keyboard
ID 67: 

Class ID "2" belongs to car and Class ID "7" belongs to truck

#3. Set Input and Output Video Paths

Specifying the paths to the input and output videos

In [ ]:
# Path to the input video
input_video_path = "/content/vehicle-counting.mp4"  # Replace with your video path

# Path to the output video
output_video_path = "/content/vehicle-counting-annotated.mp4"

#4. Initialize Video Capture and Writer

Setting up video capture for reading, and video writer for saving the annotated video

In [ ]:
# Create a Video Capture Object
cap = cv2.VideoCapture(input_video_path)

# Get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

print(f"Video Properties:\nWidth: {frame_width}\nHeight: {frame_height}\nFPS: {fps}\nTotal Frames: {total_frames}")

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # 'mp4v' is generally reliable for MP4
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

Video Properties:
Width: 3840
Height: 2160
FPS: 25.0
Total Frames: 538


#5. Initialize Counters and Tracking Structures

Setting up counters for categorizing counter based on direction and vehicle type

In [ ]:
# Initialize vehicle counters and line information
car_count_in = 0
car_count_out = 0
truck_count_in = 0
truck_count_out = 0
line_position = None

# Store tracked object IDs and their previous positions
crossed_ids = set()
track_positions = {}

#6. Define Helper Functions

Implementing the functions to draw the counting line, determining movement direction, and filtering desired classes based on class IDs (cars and trucks in this case)

In [ ]:
# Function to draw the counting line
def draw_counting_line(frame):
    global line_position
    height, width = frame.shape[:2]
    if line_position is None:  # Set the line position only once
        line_position = height - 400  # Line 400 pixels from the bottom
    cv2.line(frame, (0, line_position), (width, line_position), (0, 255, 0), 2)

# Function to determine direction
def get_direction(track_id, current_center_y):
    previous_center_y = track_positions.get(track_id, None)
    if previous_center_y is None:
        return None  # Not enough information to determine direction
    if previous_center_y < line_position and current_center_y > line_position:
        return "down"  # Moving downwards (e.g., Out)
    elif previous_center_y > line_position and current_center_y < line_position:
        return "up"    # Moving upwards (e.g., In)
    else:
        return None  # No crossing or moving parallel

# Function to check if the class ID is desired (2 for car, 7 for truck)
def is_desired_class_id(class_id):
    desired_class_ids = [2, 7]  # 2: car, 7: truck
    return class_id in desired_class_ids

#7. Processing Video Frames

This section processes each frame, performs detection and tracking: then updates counts based on direction, and finally, annotates the frames

In [ ]:
# Process video frames
frame_number = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break  # End of video

    frame_number += 1

    # Draw the counting line
    draw_counting_line(frame)

    # Run YOLO detection with ByteTrack tracking
    try:
        results = model.track(
            source=frame,
            persist=True,
            tracker="bytetrack.yaml",  # Ensure 'bytetrack.yaml' is present in the environment
            conf=0.25,  # Confidence threshold
            iou=0.5     # IoU threshold
        )
    except Exception as e:
        print(f"Error during model tracking at frame {frame_number}: {e}")
        continue  # Skip this frame

    # Process each detection
    if results[0].boxes:  # Ensure there are detections
        for box in results[0].boxes:
            # Extract bounding box, class ID, and unique track ID
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            class_id = int(box.cls)
            track_id = int(box.id)

            # Skip unwanted classes based on class ID
            if not is_desired_class_id(class_id):
                continue  # Skip this detection

            # Get the center point of the bounding box
            center_x = (x1 + x2) // 2
            center_y = (y1 + y2) // 2

            # Determine direction based on previous position
            direction = get_direction(track_id, center_y)

            # Update the previous position
            track_positions[track_id] = center_y

            # Check if the object has crossed the line and hasn't been counted yet
            if track_id not in crossed_ids and direction is not None:
                crossed_ids.add(track_id)  # Mark object as counted
                if class_id == 2:  # Car
                    if direction == "down":
                        car_count_out += 1
                    elif direction == "up":
                        car_count_in += 1
                elif class_id == 7:  # Truck
                    if direction == "down":
                        truck_count_out += 1
                    elif direction == "up":
                        truck_count_in += 1

            # Assign class name based on class ID
            if class_id == 2:
                class_name = "Car"
                color = (255, 0, 0)  # Blue for cars
            elif class_id == 7:
                class_name = "Truck"
                color = (0, 0, 255)  # Red for trucks
            else:
                class_name = "Unknown"
                color = (0, 255, 255)  # Yellow for other desired classes

            # Draw bounding box and label on the frame
            label = f"{class_name} ID:{track_id}"
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(frame, label, (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

    # Display the vehicle counts on the frame
    cv2.putText(frame, f"Cars In: {car_count_in}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
    cv2.putText(frame, f"Cars Out: {car_count_out}", (10, 70),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
    cv2.putText(frame, f"Trucks In: {truck_count_in}", (10, 110),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    cv2.putText(frame, f"Trucks Out: {truck_count_out}", (10, 150),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # Write the annotated frame to the output video
    out.write(frame)

    # Optional: Print progress every 100 frames
    if frame_number % 100 == 0:
        print(f"Processed {frame_number}/{total_frames} frames.")

print("Processing complete.")

# Release video writer and capture
out.release()
cap.release()

requirements: Ultralytics requirement ['lap>=0.5.12'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 24.5 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 1.8s, installed 1 package: ['lap>=0.5.12']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect


0: 384x640 4 cars, 1 truck, 716.0ms
Speed: 2.5ms preprocess, 716.0ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 1 truck, 605.3ms
Speed: 2.4ms preprocess, 605.3ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 1 truck, 595.8ms
Speed: 2.3ms preprocess, 595.8ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 1 truck, 569.8ms
Speed: 2.4ms preprocess, 569.8ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 1 truck, 625.9ms
Speed: 2.5ms preprocess, 625.9ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)



#8. Creating a Download Link for the Output Video

After processing, a download link i generated to retrieve the annotated video

In [ ]:
# Function to create a download link in Colab
def create_download_link(filename, title="Download Video"):
    from IPython.display import HTML
    import urllib

    if not os.path.exists(filename):
        print(f"File {filename} does not exist.")
        return

    # Encode the file to be URL-safe
    encoded = urllib.parse.quote(filename)
    href = f'<a href="{encoded}" download="{os.path.basename(filename)}">{title}</a>'
    return HTML(href)

# Display the download link
create_download_link(output_video_path)

# The final annotated video has successfully detected, tracked and counted all the cars and trucks: the dedicated IDs for each vehicle is also consistent throughout the video, further showcasing the yolov11x's capability to perform complete Computer Vision project with built-in tools and libraries.